# Showcase the *stravaio* library
## Objectives
1. Fetch your Strava data using the stravaio
2. Use plotly to plot the fancy power analysis

In [2]:
%load_ext autoreload
%autoreload 2

import stravaio
from stravaio import StravaIO, strava_oauth2
import numpy as np
import pandas as pd
from plotly.offline import iplot, init_notebook_mode
import plotly
import plotly.graph_objs as go
import plotly_express as px
init_notebook_mode(connected=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Manage Strava Access Token

To get one, head to the [strava-oauth](https://www.strava.com/oauth/authorize?client_id=8798&response_type=code&redirect_uri=http%3A%2F%2Fvelometria.com%2Fstrava-oauth%2Fauthorization_successful&scope=read%2Cprofile%3Aread_all%2Cactivity%3Aread&state=https%3A%2F%2Fgithub.com%2Fsladkovm%2Fstrava-oauth&approval_prompt=force)

Explanation: it is a running a public server https://github.com/sladkovm/strava-oauth

In [ ]:
strava_oauth2()

In [ ]:
import os
import dotenv
dotenv.load_dotenv()
access_token = os.getenv('ACCESS_TOKEN')
if access_token is None:
    raise ValueError("access_token is None! Add it to the .env file or to the system environment variable!")

## Initialize StravaIO client and start exploring your own data

In [3]:
client = StravaIO(access_token)

## Get an athlete a.k.a yourself

In [ ]:
athlete = client.get_logged_in_athlete()
athlete

## List activities summaries from the given date

In [7]:
# Returns a list of summary activities
activities = client.get_logged_in_athlete_activities(after='last week')

Fetched 4, the latests is on 2019-06-11 09:59:07+00:00


### print the names of the activities

In [8]:
[a.name for a in activities]

['G.O.A.T',
 'With Antonio',
 'Afternoon Ride',
 'SUF Strength - Intermediate Level 3A']

## get the full activity of interest

In [10]:
a = client.get_activity_by_id(id=activities[-2].id)
a

Activity: 2436304110, Date: 2019-06-09 11:13:18+00:00, Name: Afternoon Ride

## get associated stream

In [15]:
s = client.get_activity_streams(activities[-2].id, athlete_id=athlete.id)
s

Streams for 2436304110
Keys: ['time', 'distance', 'altitude', 'velocity_smooth', 'heartrate', 'cadence', 'watts', 'temp', 'moving', 'grade_smooth', 'lat', 'lng']
Access: obj.key or obj.to_dict() to load into a pd.DataFrame()

## With activity and stream in place let's lounch pandas and plotly

In [16]:
df = pd.DataFrame(s.to_dict())
df.head()

,time,distance,altitude,velocity_smooth,heartrate,cadence,watts,temp,moving,grade_smooth,lat,lng
0,165,458.7,389.6,5.0,113,82,131,26,True,3.2,47.084871,15.442007
1,166,463.3,389.8,4.9,113,80,165,26,True,3.4,47.084894,15.442057
2,167,467.7,390.0,4.8,113,78,168,26,True,4.7,47.084916,15.442106
3,168,471.7,390.2,4.5,113,77,178,26,True,3.6,47.084932,15.442152
4,169,475.8,390.4,4.3,113,78,177,26,True,2.5,47.084949,15.442200


## Let's add a color to the DataFrame
* white - for low intensity
* grey - for moderate intensity
* orange - for high intensity

In [19]:
ftp=athlete.api_response.ftp
strava_orange = '#fc4c02'
df['color']= 'white'
df.loc[df['watts'] > ftp*0.7, 'color'] = 'grey'
df.loc[df['watts'] > ftp*1.0, 'color'] = strava_orange
df.head()

,time,distance,altitude,velocity_smooth,heartrate,cadence,watts,temp,moving,grade_smooth,lat,lng,color
0,165,458.7,389.6,5.0,113,82,131,26,True,3.2,47.084871,15.442007,white
1,166,463.3,389.8,4.9,113,80,165,26,True,3.4,47.084894,15.442057,white
2,167,467.7,390.0,4.8,113,78,168,26,True,4.7,47.084916,15.442106,white
3,168,471.7,390.2,4.5,113,77,178,26,True,3.6,47.084932,15.442152,grey
4,169,475.8,390.4,4.3,113,78,177,26,True,2.5,47.084949,15.442200,grey


## Let's create a histogram of power with bin size 25 watts

In [33]:
groups, bins = pd.cut(df.watts, bins=np.arange(0, df.watts.max(), 10), retbins=True)
hist = df['watts'].groupby(groups).count()
hist

watts
(0, 10]        72
(10, 20]       50
(20, 30]       25
(30, 40]       22
(40, 50]       24
(50, 60]       27
(60, 70]       44
(70, 80]      121
(80, 90]      196
(90, 100]     217
(100, 110]    199
(110, 120]    208
(120, 130]    180
(130, 140]    208
(140, 150]    214
(150, 160]    218
(160, 170]    202
(170, 180]    194
(180, 190]    237
(190, 200]    248
(200, 210]    234
(210, 220]    336
(220, 230]    346
(230, 240]    360
(240, 250]    300
(250, 260]    187
(260, 270]    107
(270, 280]     75
(280, 290]     65
(290, 300]     60
(300, 310]     35
(310, 320]     21
(320, 330]     15
(330, 340]     11
(340, 350]      8
(350, 360]      2
(360, 370]      5
(370, 380]      1
(380, 390]      3
(390, 400]      1
(400, 410]      1
(410, 420]      1
(420, 430]      3
(430, 440]      1
(440, 450]      0
(450, 460]      0
Name: watts, dtype: int64

## Let's create colors for the bins in the same way as we did for the raw power

In [34]:
bin_c = pd.DataFrame({'watts': bins})
bin_c['color'] = 'white'
bin_c.loc[bin_c['watts'] > 0.6*ftp, 'color'] = 'grey'
bin_c.loc[bin_c['watts'] > 0.9*ftp, 'color'] = strava_orange

# Plotly time

In [46]:
altitude = go.Scatter(
    x=df.time,
    y=df.altitude/df.altitude.max()*df.watts.max(),
    fill='tozeroy',
    fillcolor='rgb(30,30,30)',
    line=dict(
        color='rgb(30,30,30)'
    )
)

watts = go.Scattergl(
    x=df.time,
    y=df.watts,
    mode='markers',
    marker=dict(
        color=df['color'],
        size=2
    )
)


watts_b = go.Bar(
    y = bins,
    x = hist,
    orientation='h',
    marker=dict(
        color = bin_c['color']
    )
)



fig = plotly.tools.make_subplots(rows=1, cols=2, shared_yaxes=True)

fig.append_trace(altitude, 1,1)
fig.append_trace(watts, 1,1)
fig.append_trace(watts_b, 1,2)
fig.layout.update(
    autosize=False,
    width=800,
    height=300,
    margin=go.layout.Margin(
        l=50,
        r=20,
        b=10,
        t=30,
        pad=4
    ),
)
fig.layout.update(plot_bgcolor='black')
fig.layout.update(xaxis=dict(domain=[0, 0.8], showticklabels=False))
fig.layout.update(xaxis2=dict(domain=[0.8, 1.0], showticklabels=False))
fig.layout.update(yaxis=dict(showline=False, showticklabels=False, title='Power & Altitude a.u.'))
fig.layout.update(bargap=0)
fig.layout.update(showlegend=False)
fig.layout.update(title='Strava ride replotted using stravaio and plotly')

iplot(fig)

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y1 ]

